In [1]:
import pickle
import matplotlib.pyplot as plt

from classifier import *
from preprocessing import *
from staff_removal import *
from helper_methods import *

In [2]:
# Get paths of scaned and captured test cases #
scanned_path = './test-cases/test-set-scanned/scanned'
captured_path = './test-cases/test-set-camera-captured/captured'

img_filenames = os.listdir(scanned_path)
for i, fn in enumerate(img_filenames):
    test_number = fn.split('.')[0]

In [3]:
def preprocess_img(img_path, output_path):
    # 1. Read desired image #
    img = cv2.imread(img_path, 0)
    
    # 2. Remove noise (odd pixels) from the image and save it #
    img = cv2.fastNlMeansDenoising(img, None, 10, 7, 21)
    cv2.imwrite('testing-output/{}/1. noise_removed.png'.format(output_path), img)

    # 3. Binarize image using combination of (global + otsu) thresholding and save it #
    threshold, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2.imwrite('testing-output/{}/2. binarized.png'.format(output_path), img)

    # 4. Return image shape (width, height) and processed image # 
    n, m = img.shape
    return n, m, img

## Steps of processing
1. ~Apply Pre-processing~
2. ~Remove Staff lines~
3. ~Cut images into buckets~
4. ~Get reference line for each bucket~
5. ~Segment symbols, Sort them by x-value~
6. Classify symbol

## Loading dataset

In [4]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset('hog')
print('Finished loading dataset.')

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=random_seed)

Loading dataset. This will take time ...
finished processing:  a_1
finished processing:  a_16
finished processing:  a_16_flipped
finished processing:  a_2
finished processing:  a_2_flipped
finished processing:  a_32
finished processing:  a_4
finished processing:  a_4_flipped
finished processing:  a_8
finished processing:  a_8_flipped
finished processing:  b
finished processing:  barline
finished processing:  b_16
finished processing:  b_16_flipped
finished processing:  b_8
finished processing:  b_8_flipped
finished processing:  chord_2
finished processing:  chord_3
finished processing:  chord_3_2
finished processing:  chord_special
finished processing:  Clef
finished processing:  d
finished processing:  dot
finished processing:  hash
finished processing:  symbol_bb
finished processing:  t_2
finished processing:  t_4
finished processing:  x
Finished loading dataset.


## Training and saving the model

In [5]:
model_name = 'SVM'
model = run_experiment(train_features, test_features, train_labels, test_labels, model_name)

filename = f'{model_name}.sav'
pickle.dump(model, open(filename, 'wb'))
 
# TODO: Use this line to load the model 
# model = pickle.load(open(filename, 'rb'))

############## Training SVM ##############
SVM accuracy: 99.40334128878283 %


## Testing 

In [6]:
# path = './data-set/a_4'

# img_filenames = os.listdir(path)
# for i, fn in enumerate(img_filenames):
#     if fn != '':
#         img = cv2.imread(f'{path}/{fn}', 0)
#         img = clean_and_cut(img)
#         cv2.imwrite(f'{path}/{fn}', img)

In [9]:
direct_labels = ['x', 'b', 'Clef', 'dot', 'hash', 'd', 't_2', 't_4', 'symbol_bb', 'barline']

direct_texts = {'x':'##', 'b':'&', 'hash':'#', 'd':'', 'symbol_bb':'&&', 'dot':'.', 'Clef':'', 't_2':'2', 't_4':'4', 'barline':''}

def get_a_character(distance, line_spacing, flipped = 0):
    # TODO: change it to chars array manipluation 
    # TODO: check 4.5

    if flipped and distance < 2.75 * line_spacing:
        return 'b'
    if distance < flipped * (4.5 * line_spacing) + .25 * line_spacing:
        return 'c'
    if distance < flipped * (4.5 * line_spacing) + .75 * line_spacing:
        return 'd'
    if distance < flipped * (4.5 * line_spacing) + 1.25 * line_spacing:
        return 'e'
    if distance < flipped * (4.5 * line_spacing) + 1.75 * line_spacing:
        return 'f'
    if distance < flipped * (4.5 * line_spacing) + 2.25 * line_spacing:
        return 'g'
    if distance < flipped * (4.5 * line_spacing) + 2.75 * line_spacing:
        return 'a'
    if distance < flipped * (4.5 * line_spacing) + 3.25 * line_spacing:
        return 'b'


def text_operation(label, ref_line, line_spacing, y1, y2): 
    if label in direct_labels:
        return direct_texts[label]
        
    if not(label.endsWith('flipped')): 
        distance = 0
        if label.startsWith('a_'):
            distance = ref_line - y2
            character = get_a_character(distance, line_spacing)

        if label == 'a_1':
            return character + '/1'
        if label == 'a_2':
            return character + '/2'
        if label == 'a_4':
            return character + ''
        if label == 'a_8':
            return character + '/8'
        if label == 'a_16':
            return character + '/16'
        if label == 'a_32':
            return character + '/32'
    else: # flipped
        distance = 0
        if label.startsWith('a_'):
            distance = ref_line - y1
            character = get_a_character(distance, line_spacing, 1)

        if label.startsWith('a_2'):
            return character + '2/2'
        if label.startsWith('a_4'):
            return character + '2'
        if label.startsWith('a_8'):
            return character + '8/2'
        if label.startsWith('a_16'):
            return character + '16/2'
        if label.startsWith('a_32'):
            return character + '32/2'


IndentationError: expected an indented block (<ipython-input-9-f87bc3958256>, line 4)

In [46]:
def cut_boundaries(cur_symbol, no_of_cuts, y2):
    last_x = 0
    cutted_boundaries = []
    step = cur_symbol.shape[1] // no_of_cuts

    for i in range(no_of_cuts):
        cur = cur_symbol[:, last_x:last_x + step]
        last_x += step

        white = np.argwhere(cur == 255)
        white = sorted(white, key=lambda x: x[0])

        y_min, y_max = white[0][0], white[-1][0]
        ret_boundary = [0, y_min, cur.shape[1] - 1, y_max]
        
        diff_y1, diff_y2 = cur.shape[0] - y_min, cur.shape[0] - y_max
        ret_boundary[1] = y2 - diff_y1
        ret_boundary[3] = y2 - diff_y2
        
        cutted_boundaries.append(ret_boundary)

    return cutted_boundaries

In [7]:
# Threshold for line to be considered as an initial staff line #
cnt = 0
threshold = 0.8
correct_labels = np.load('correct_labels.npy')

for i in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']:
    try: 
        os.mkdir('testing-output/{}'.format(i)) 
    except OSError as error: 
        pass

    f = open("testing-output/{i}.txt", "w")

    # Get image and its dimensions#
    height, width, in_img = preprocess_img('{}/{}.png'.format(scanned_path, i), '{}'.format(i))
    
    # Get line thinkness and list of staff lines #
    staff_lines_thicknesses, staff_lines = get_staff_lines(width, height, in_img, threshold)

    # Remove staff lines from original image #
    cleaned = remove_staff_lines(in_img, width, staff_lines, staff_lines_thicknesses)
    cv2.imwrite('testing-output/{}/3. cleaned.png'.format(i), cleaned)
    
    # Get list of cutted buckets and cutting positions #
    cut_positions, cutted = cut_image_into_buckets(cleaned, staff_lines)
    
    # Get reference line for each bucket #
    ref_lines, lines_spacing = get_ref_lines(cut_positions, staff_lines)
    
    for it in range(len(cutted)):
        cur_img = cutted[it].copy()
        symbols_boundries = segmentation(cutted[it])
        symbols_boundries.sort(key = lambda x: (x[0], x[1]))
        
        symbols = []
        for boundry in symbols_boundries:
            # Get the current symbol #
            x1, y1, x2, y2 = boundry
            cur_symbol = cutted[it][y1:y2+1, x1:x2+1]
            
            # Clean and cut #
            cur_symbol = clean_and_cut(cur_symbol)
            cur_symbol = 255 - cur_symbol

            # Start prediction of the current symbol #
            feature = extract_features(cur_symbol, 'hog')
            label = model.predict([feature])
            
            if label == 'b_8' or label == 'b_8_flipped':
                cutted_boundaries = cut_boundaries(cur_symbol, 2)
            elif label == 'b_16' or label == 'b_16_flipped':
                cutted_boundaries = cut_boundaries(cur_symbol, 4)
            else 
                cutted_boundaries = cut_boundaries(cur_symbol, 1)
 
            for cut_boundary in cut_boundaries:
                ## TODO: text operations -> parameters(label, distance_to_the_ref_line => ref_line[it] - y2)
                _, y1, _, y2 = cutted_boundary
                text = text_operation(label, ref_lines[it], lines_spacing[it], y1, y2)
                f.write(text)


            if correct_labels[cnt] != label:    
                print(cnt, label)
            
                plt.axis('off')
                io.imshow(cur_symbol)
                io.show()
                
            cnt+=1

        f.write('\n')
    
        cv2.imwrite(f'testing-output/{i}/4. cutted-{it + 1}.png', cutted[it])
        cv2.imwrite(f'testing-output/{i}/5. cutted-segmented-{it + 1}.png', cur_img)

    
    f.close()        